In [1]:
import numpy as np
import pandas as pd
import cv2
from keras.models import Sequential

In [25]:
use_cache = 1
dataset_path = r'../Distracted-Driver-Detection/Dataset/'

In [2]:
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(src=img, dsize=(224, 224), interpolation=cv2.INTER_LINEAR)
    return resized

In [20]:
def load_train():
    df = pd.read_csv(dataset_path + 'auc.distracted.driver.train.csv')
    x = df.iloc[:,0]
    y = df.iloc[:,1]
    X_train = []
    Y_train = []
    print('Read train images')
    for i in range (0,len(x)):
        fl= dataset_path + r'v1_cam1_no_split/' + r'/'.join(x[i].split('/')[-2:])
        print(fl)
        img = get_im_cv2(fl)
        X_train.append(img)
        Y_train.append(y[i])
    return X_train, Y_train

In [27]:
def load_valid():
    df = pd.read_csv(dataset_path + 'auc.distracted.driver.test.csv')
    x = df.iloc[:,0]
    y = df.iloc[:,1]
    X_valid = []
    Y_valid = []
    print('Read test images')
    for i in range (0,len(x)):
        fl = dataset_path + r'v1_cam1_no_split/' + r'/'.join(x[i].split('/')[-2:])
        print(fl)
        img = get_im_cv2(fl)
        X_valid.append(img)
        Y_valid.append(y[i])
    return X_valid, Y_valid

In [ ]:
X_valid, Y_valid = load_valid()